In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

import pandas as pd

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-medium"
model_gen = "model_gen2"
data_gen = "data_gen2"

In [ ]:
model = GPT2LMHeadModel.from_pretrained(f'/content/drive/MyDrive/Colab Notebooks/{model_gen}/')
tokenizer = GPT2TokenizerFast.from_pretrained(f'/content/drive/MyDrive/Colab Notebooks/{model_gen}/')
model.to(device)


In [ ]:
unique_sentences = set()
unique_words = set()

for i in [0,1,2,3,4,5,6,7,8,9]:

  file_path = f'/content/drive/MyDrive/Colab Notebooks/{data_gen}/results_{i}.txt'

  with open(file_path, "r") as file:
      for line in file:
          sentence = line.replace("Prompt: ", "").strip()
          if len(sentence)>0:
            unique_sentences.add(sentence)
            
            words = sentence.split()
            unique_words.update(words)

print(f"Number of unique sentences: {len(unique_sentences)}")
print(f"Number of unique words: {len(unique_words)}")

In [ ]:
for i in [0,1,2,3,4,5,6,7,8,9]:

    file_path = f'/content/drive/MyDrive/Colab Notebooks/{data_gen}/results_{i}.txt'

    with open(file_path, "r") as file:
        for line in file:
            sentence = line.replace("Prompt: ", "").strip()
            
            # Add the sentence to the set of unique sentences
            total_sentences.append(sentence)

words = [word.lower() for sentence in total_sentences for word in sentence.split()]
lexical_diversity = len(set(words)) / len(words)
print(f"Lexical Diversity {data_gen}: {lexical_diversity}")


In [ ]:
import textstat

def compute_overall_readability(sentences):
    total_flesch_reading_ease = 0
    total_flesch_kincaid_grade = 0
    total_gunning_fog_index = 0

    for sentence in sentences:
        total_flesch_reading_ease += textstat.flesch_reading_ease(sentence)
        total_flesch_kincaid_grade += textstat.flesch_kincaid_grade(sentence)
        total_gunning_fog_index += textstat.gunning_fog(sentence)

    num_sentences = len(sentences)
    overall_flesch_reading_ease = total_flesch_reading_ease / num_sentences
    overall_flesch_kincaid_grade = total_flesch_kincaid_grade / num_sentences
    overall_gunning_fog_index = total_gunning_fog_index / num_sentences

    return {
        "Overall Flesch Reading Ease": overall_flesch_reading_ease,
        "Overall Flesch-Kincaid Grade": overall_flesch_kincaid_grade,
        "Overall Gunning Fog Index": overall_gunning_fog_index
    }

total_sentences = []
for i in [0,1,2,3,4,5,6,7,8,9]:

    file_path = f'/content/drive/MyDrive/Colab Notebooks/{data_gen}/results_{i}.txt'

    # Process the text file
    with open(file_path, "r") as file:
        for line in file:
            sentence = line.replace("Prompt: ", "").strip()
            
            total_sentences.append(sentence)


# Compute overall readability
overall_scores = compute_overall_readability(total_sentences)
print(f"Iteration {data_gen}: {overall_scores}")

In [ ]:
from textblob import TextBlob

for data in ['data_gen2']:
  total_sentences = []
  for i in [0,1,2,3,4,5,6,7,8,9]:

    file_path = f'/content/drive/MyDrive/Colab Notebooks/{data}/results_{i}.txt'

    # Process the text file
    with open(file_path, "r") as file:
        for line in file:
            sentence = line.replace("Prompt: ", "").strip()
            
            # Add the sentence to the set of unique sentences
            total_sentences.append(sentence)
  polarity = []
  for sentence in total_sentences[:455]:
    if len(sentence)> 0:
      blob = TextBlob(sentence)
      polarity.append(blob.sentiment.polarity)

  mean_polarity = sum(polarity) / len(polarity)
  print(f'{data} polarity: {mean_polarity}')